In [20]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
import json
import re

In [ ]:
parser = LlamaParse(api_key="YOUR LLAMA CLOUD API KEY", result_type="markdown", spreadsheet_extract_sub_tables=True)
documents = SimpleDirectoryReader(input_dir="./data", file_extractor={".xls": parser}).load_data()
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents(documents)

Started parsing the file under job_id ba46320c-312a-4340-859a-bac5b4e8c741


In [ ]:
llm = Groq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key="YOUR GROQ API KEY",
    temperature=0.1,
)

In [23]:
query_engine = index.as_query_engine(llm=llm)

In [ ]:
def list_tables():
    query = "Just print all the tables names in an array format. Don't include any other text in your response."
    response = query_engine.query(query)
    return {"tables": eval(str(response))}

def get_table_details(table_name: str):
    query = f"""For the table named "{table_name}", just return the row names that appear in the first column. 
Return them as a JSON array under the key "row_names" only. Do not include any explanation."""
    response = query_engine.query(query)
    return {"table_name": table_name, "row_names": eval(str(response))}


def extract_json_from_response(text):
    matches = re.findall(r'\{.*?\}', str(text), re.DOTALL)
    for match in reversed(matches):  # Try from last to first
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue
    print("No valid JSON found in response.")
    print(text)
    return {}

def row_sum(table_name: str, row_name: str):
    query = f"""In the table named "{table_name}", locate the row titled "{row_name}" and sum all the numeric values in that row. 
Only return JSON in the following format:
{{"table_name": "{table_name}", "row_name": "{row_name}", "sum": [SUM_HERE]}}"""
    response = query_engine.query(query)
    return extract_json_from_response(response)


In [26]:
print("=== List Tables ===")
print(list_tables())

=== List Tables ===
{'tables': ['CapBudgWS', 'Investment Measures', 'BOOK VALUE & DEPRECIATION']}


In [27]:
print("\n=== Table Details: Initial Investment ===")
print(get_table_details("Initial Investment"))


=== Table Details: Initial Investment ===
{'table_name': 'Initial Investment', 'row_names': {'row_names': ['Investment', '- Tax Credit', 'Net Investment', '+ Working Cap', '+ Opp. Cost', '+ Other invest.', 'Initial Investment']}}


In [28]:
print("\n=== Row Sum: 'Tax Credit (if any )=' in 'Initial Investment' ===")
print(row_sum("Initial Investment", "Tax Credit (if any )="))


=== Row Sum: 'Tax Credit (if any )=' in 'Initial Investment' ===
{'table_name': 'Initial Investment', 'row_name': 'Tax Credit (if any )=', 'sum': [10]}
